In [ ]:
import keras
import matplotlib.pyplot as plt

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

## Load images from directory

In [ ]:
!gdown 1JFZDEqRcReAjJiqCxz-W4707ipDEFe6i -O trainset.zip
!unzip -q trainset.zip -d trainset/

In [ ]:
train_image_dir = '/content/trainset/MultiClassDataDemo/'

In [ ]:
train_generator = train_datagen.flow_from_directory(train_image_dir,
                                                    target_size=(256, 256),
                                                    batch_size=64,
                                                    class_mode='categorical')

In [ ]:
train_generator.class_indices

In [ ]:
image_shape = train_generator.image_shape

In [ ]:
train_generator.n

## Read the Images

In [ ]:
x_batch, y_batch = next(train_generator)

In [ ]:
x_batch.shape

In [ ]:
y_batch.shape

In [ ]:
plt.imshow( x_batch[0] );
plt.grid(False)    
plt.show();

### Show random images from a batch

In [ ]:
import random 

def show_random_images( x_batch_images, y_batch_images ):

    all_indexes = list(range(len(x_batch_images)))
    random_indexes = random.sample( all_indexes, 8 )

    plt.figure( figsize=(16, 8))
    
    k = 1    
    for i in random_indexes:
        plt.subplot(2, 4, k);
        plt.grid(False)
        plt.imshow(x_batch_images[i])
        plt.title(y_batch_images[i])
        k = k + 1

    plt.show()

In [ ]:
show_random_images( x_batch, y_batch )

## Image Rotations

The training dataset contains images, which are mostly aligned vertically. But while predicting users may take leaf images from many different angles. So we augment our training samples by randomly rotating images from 0 to 90 degrees.

In [ ]:
rotate_train_datagen = ImageDataGenerator(rotation_range=90, 
                                          rescale=1./256)

rotate_train_generator = rotate_train_datagen.flow_from_directory(train_image_dir,
                                                    target_size=(256, 256),
                                                    batch_size=32,
                                                    class_mode='categorical')

In [ ]:
x_rotate_batch, y_batch = next(rotate_train_generator)

In [ ]:
x_rotate_batch.shape

In [ ]:
show_random_images(x_rotate_batch, y_batch)

## Zooming In and Out

In [ ]:
zoom_train_datagen = ImageDataGenerator(zoom_range=0.4, 
                                        rescale=1./256)

zoom_train_generator = zoom_train_datagen.flow_from_directory(train_image_dir,
                                                    target_size=(256, 256),
                                                    batch_size=32,
                                                    class_mode='categorical')

In [ ]:
x_zoom_batch, y_batch = next(zoom_train_generator)
show_random_images(x_zoom_batch, y_batch)

## Shifting Images

In [ ]:
shift_train_datagen = ImageDataGenerator(rescale=1./256,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2)

shift_train_generator = shift_train_datagen.flow_from_directory(train_image_dir,
                                                    target_size=(256, 256),
                                                    batch_size=32,
                                                    class_mode='categorical')

In [ ]:
x_shift_batch, y_batch = next(shift_train_generator)
show_random_images(x_shift_batch, y_batch)

## Combining multiple data augmentation techniques

In [ ]:
augmented_train_datagen = ImageDataGenerator(rescale=1./256,
                                         rotation_range=90,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2,
                                         zoom_range=0.4)

augmented_train_generator = augmented_train_datagen.flow_from_directory(train_image_dir,
                                                    target_size=(256, 256),
                                                    batch_size=128,
                                                    class_mode='categorical')

In [ ]:
x_augmented_batch, y_batch = next(augmented_train_generator)
show_random_images(x_augmented_batch, y_batch)

## Building a model

In [ ]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, Input, ReLU

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=1, padding='same', input_shape=image_shape))
model.add(ReLU())
                            
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same', input_shape=image_shape))
model.add(ReLU())
                            
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(filters=64, 
                 kernel_size=(3,3), 
                 strides=1, 
                 padding='same', 
                 input_shape=image_shape))
model.add(ReLU())
                            
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Flatten())
    
model.add(Dense(256))
model.add(ReLU())

model.add(Dense(64))
model.add(ReLU())

model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
callbacks_list = [ReduceLROnPlateau(monitor='val_loss',
                                    factor=0.1, 
                                    patience=2),
                 EarlyStopping(monitor='val_loss',
                               patience=2),
                 ModelCheckpoint(filepath='my_model.h5',
                                 monitor='val_loss',
                                 save_best_only=True)]

In [ ]:
%%time

history = model.fit_generator(augmented_train_generator,
                              steps_per_epoch=20,
                              epochs=5,
                              callbacks=callbacks_list,
                              validation_data=augmented_train_generator,
                              validation_steps=5)

In [ ]:
import pickle

pickle.dump( history.history, open('history_aug_nozca.pkl', 'wb') )

In [ ]:
# summarize history for accuracy

def plot_train_val_accuracy(hist):
    plt.plot(hist['acc'])
    plt.plot(hist['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
# summarize history for loss

def plot_train_val_loss(hist):
    plt.plot(hist['loss'])
    plt.plot(hist['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
plot_train_val_accuracy( history.history )

In [ ]:
history.history.values()

## ZCA Whiteing

A whitening transform minimizes the redundancy in the matrix of pixel images. Initutively, it highlights the pixels with high variance across images.  

Image whitening uses the same dimesional reduction technique as Principal Component Analysis (PCA), alternative called ZCA. However the transformed image retains the same dimensions of original image.

You can perform a ZCA whitening transform by setting the zca_whitening argument to True in *ImageDataGenerator*.

In [ ]:
zca_train_datagen = ImageDataGenerator(zca_whitening=True)

zca_train_generator = zca_train_datagen.flow_from_directory(train_image_dir,
                                                    target_size=(256, 256),
                                                    batch_size=8,
                                                    class_mode='categorical')

In [ ]:
x_zca_batch, y_batch = next(zca_train_generator)

In [ ]:
x_zca_batch.shape

In [ ]:
show_random_images( x_zca_batch, y_batch )

## Combined Data Augmentation + whitening

In [ ]:
augmented_zca_train_datagen = ImageDataGenerator(rotation_range=90,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2,
                                         zoom_range=0.4,
                                         zca_whitening=True)

augmented_zca_train_generator = augmented_zca_train_datagen.flow_from_directory(train_image_dir,
                                                    target_size=(256, 256),
                                                    batch_size=128,
                                                    class_mode='categorical')

In [ ]:
x_augmented_zca_batch, y_batch = next(augmented_zca_train_generator)
show_random_images(x_augmented_zca_batch, y_batch)

In [ ]:
K.clear_session()  # clear default graph

model_zca = Sequential()
model_zca.add(Conv2D(filters=16, kernel_size=(3,3), strides=1, padding='same', input_shape=image_shape))
model_zca.add(LeakyReLU(0.1))
                            
model_zca.add(MaxPooling2D(pool_size=(3, 3)))

model_zca.add(Conv2D(filters=32, kernel_size=(3,3), strides=1, padding='same', input_shape=image_shape))
model_zca.add(LeakyReLU(0.1))
                            
model_zca.add(MaxPooling2D(pool_size=(3, 3)))

model_zca.add(Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', input_shape=image_shape))
model_zca.add(LeakyReLU(0.1))
                            
model_zca.add(MaxPooling2D(pool_size=(3, 3)))

model_zca.add(Flatten())
    
model_zca.add(Dense(256))
model_zca.add(LeakyReLU(0.1))
model_zca.add(Dropout(0.5))

model_zca.add(Dense(64))
model_zca.add(LeakyReLU(0.1))
model_zca.add(Dropout(0.5))

model_zca.add(Dense(10))
model_zca.add(Activation('softmax'))

In [ ]:
model_zca.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
callbacks_list = [ReduceLROnPlateau(monitor='val_loss',
                                    factor=0.1, 
                                    patience=2),
                 EarlyStopping(monitor='val_loss',
                               patience=2),
                 ModelCheckpoint(filepath='my_model.h5',
                                 monitor='val_loss',
                                 save_best_only=True)]

In [ ]:
%%time

history_zca = model_zca.fit_generator(augmented_train_generator,
                              steps_per_epoch=100,
                              epochs=50,
                              callbacks=callbacks_list,
                              validation_data=augmented_train_generator,
                              validation_steps=5)

In [ ]:
pickle.dump( history.history, open('history_aug_zca.pkl', 'wb') )

In [ ]:
plot_train_val_accuracy(history_zca.history)